In [1]:
from ccdc import io, search
from ccdc.interaction import InteractionLibrary
import numpy as np
import matplotlib.pyplot as plt
import time
import csv
import glob
import os
import re
import pywindow as pw
from ase.io import read, write
from ase.visualize import view
from ase.atoms import Atom, Atoms
from molmass import Formula
import pandas as pd
from string import ascii_uppercase

In [2]:
def f_to_str(formula):
    f1 = str(formula) #intakes str of chem formula
    f2 = formula.split() # splits into list
    
    f3 = [i.split('.', 1)[0] for i in f2] # splits off atom fractions and deletes them
    
    f4 = [re.findall(r'(\w+?)(\d+)', i)[0] for i in f3] #splits element from number
    
    f5 = [] #rewrites tuples back into list form and changes 0 -> 1 to avoid undercounting elements
    for i in f4:
        i = list(i)
        if i[1] == '0':
            i[1] = '1'
        f5.append(i)
    
    f6 = ["".join(i) for i in f5] # joins the elements and numbers
    f7 = " ".join(f6) # reforms altered string in orginal format
    
    return f7

In [3]:
annotation_csv_path="mof_data.csv"
comps = "compositional_data.csv"

def write_annotations(annotation_csv_path, comps, metal_list):
    frame = []
    with open(annotation_csv_path, 'w') as writer:
        writer.write('identifier, formula, density, mass\n')
        # open(comps, 'w')
        
        csd_reader = io.EntryReader('CSD')
        for line in open("refcodes.gcd", 'r'):
            refcode = line.rstrip()
            current_crystal = csd_reader.crystal(refcode)
            current_molecule = csd_reader.molecule(refcode)
          
            avg_coord_metal = 0
            metal_coord = 0
            atom_count = 0
            print(refcode)
            for atom in current_crystal.asymmetric_unit_molecule.atoms:
                found=False
    
                if atom.atomic_number in metal_list:
                    atom_multiplicity = 0
        
                    for site in current_crystal.packing(inclusion='OnlyAtomsIncluded', box_dimensions=((0,0,0), (0.999, 0.999, 0.999))).atoms:
                        if atom.label == site.label or (atom.label == site.label[:-1] and site.label[-1] in ascii_uppercase):
                            atom_multiplicity += 1  
                    
                    atom_count += atom_multiplicity
                    
                    for molecule_atom in current_crystal.molecule.atoms:
                        if molecule_atom.fractional_coordinates == atom.fractional_coordinates:              
                            print('Atomic number', atom.atomic_number, 'has CN = ', len(molecule_atom.bonds), "and multiplicity", atom_multiplicity)
                            metal_coord += (len(molecule_atom.bonds)*atom_multiplicity)
                            # atom_count += atom_multiplicity ?where?
                            found = True

                            break
    
                    if not found:
                        print(f"Error - atom {atom} was not found in the unit cell!")
            
            if atom_count != 0:
                avg_coord_metal = metal_coord/atom_count
            else:
                avg_coord_metal = 0
            
            # write row to CSV
            formula = current_crystal.formula
            premmf = f_to_str(formula) # 
            mmf = Formula(premmf) # brings in molmass plugin for reading/processing formula
            mass = mmf.mass     
            density = current_crystal.calculated_density
                           
            df = pd.DataFrame(mmf.composition()) # sets up pandas dataframe with composition data
            df.to_csv(comps, mode='a', index=False, header=[refcode, 'X', 'X', '#'])  # writes dataframe to csv
            
            # print(premmf)
            # print(mmf.composition())
            
            writer.write('{},{},{},{},{}\n'.format( # writer adds to the data.csv
                refcode,
                premmf if premmf else '',
                density if density else '',
                mass,
                avg_coord_metal))

In [6]:
metal_list = [3, 4, 11, 12, 13, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 90, 92, 93, 94]
write_annotations(annotation_csv_path, comps, metal_list)

ABADUG
Atomic number 30 has CN =  5 and multiplicity 1
Atomic number 30 has CN =  5 and multiplicity 1
Atomic number 30 has CN =  5 and multiplicity 1
Atomic number 30 has CN =  5 and multiplicity 1
Atomic number 30 has CN =  5 and multiplicity 1
Atomic number 30 has CN =  5 and multiplicity 1
ABEDEU
Atomic number 79 has CN =  2 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 4
ABELOM01
Atomic number 29 has CN =  4 and multiplicity 8
Atomic number 29 has CN =  4 and multiplicity 16
ABEXEN
Atomic number 49 has CN =  8 and multiplicity 12
ACEMEE
Atomic number 29 has CN =  4 and multiplicity 48
Atomic number 29 has CN =  2 and multiplicity 48
Atomic number 29 has CN =  2 and multiplicity 48
ACENIF
Atomic number 30 has CN =  4 and multiplicity 16
Atomic number 30 has CN =  5 and multiplicity 16
Atomic number 30 has CN =  5 and multiplicity 16
ACIBOE
Atomic number 30 has CN =  6 

Atomic number 26 has CN =  6 and multiplicity 20
AVUYEY
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 12
AWASEY
Atomic number 59 has CN =  10 and multiplicity 8
Atomic number 29 has CN =  4 and multiplicity 8
Atomic number 29 has CN =  3 and multiplicity 4
Atomic number 29 has CN =  4 and multiplicity 8
Atomic number 29 has CN =  4 and multiplicity 8
Atomic number 29 has CN =  4 and multiplicity 8
Atomic number 29 has CN =  4 and multiplicity 8
Atomic number 29 has CN =  3 and multiplicity 4
Atomic number 19 has CN =  7 and multiplicity 4
Atomic number 59 has CN =  8 and multiplicity 4
AWATAV
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 16
AWEXUX
Atomic number 30 has CN =  5 and multiplicity 4
Atomic number 30 has CN =  5 and multiplicity 4
AXINAX01
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8

Atomic number 30 has CN =  4 and multiplicity 2
BIPLAQ
Atomic number 40 has CN =  7 and multiplicity 27
Atomic number 29 has CN =  4 and multiplicity 12
BIPXIK
Atomic number 27 has CN =  6 and multiplicity 1
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  6 and multiplicity 2
BIXVIQ
Atomic number 29 has CN =  4 and multiplicity 44
BOGNES
Atomic number 40 has CN =  8 and multiplicity 12
BOGXIF02
Atomic number 29 has CN =  3 and multiplicity 18
BOGXIF
Atomic number 29 has CN =  3 and multiplicity 18
BOLZOU
Atomic number 27 has CN =  6 and multiplicity 1
Atomic number 27 has CN =  6 and multiplicity 2
Atomic number 27 has CN =  4 and multiplicity 2
BOMWAE
Atomic number 25 has CN =  6 and multiplicity 4
Atomic number 25 has CN 

Atomic number 30 has CN =  6 and multiplicity 8
CELZUQ
Atomic number 13 has CN =  6 and multiplicity 8
Atomic number 13 has CN =  6 and multiplicity 8
CEMJOV
Atomic number 49 has CN =  7 and multiplicity 4
Atomic number 49 has CN =  6 and multiplicity 2
CEMLIS
Atomic number 13 has CN =  6 and multiplicity 6
Atomic number 13 has CN =  6 and multiplicity 6
Atomic number 13 has CN =  4 and multiplicity 3
CENQAP
Atomic number 30 has CN =  5 and multiplicity 4
Atomic number 30 has CN =  5 and multiplicity 4
Atomic number 30 has CN =  6 and multiplicity 4
Atomic number 30 has CN =  4 and multiplicity 4
CESPAU
Atomic number 48 has CN =  7 and multiplicity 2
Atomic number 48 has CN =  6 and multiplicity 2
CETKUK
Atomic number 30 has CN =  4 and multiplicity 8
Atomic number 30 has CN =  4 and multiplicity 8
CETTUT
Atomic number 30 has CN =  6 and multiplicity 8
Atomic number 30 has CN =  6 and multiplicity 8
CEZCOC
Atomic number 26 has CN =  8 and multiplicity 4
Atomic number 26 has CN =  5 and

Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  5 and multiplicity 8
CUSHIK
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  5 and multiplicity 8
CUSHUW
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  2 and multiplicity 4
CUSJAE
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  6 and multiplicity 8
Atomic number 27 has CN =  5 and multiplicity 8
CUSJOS
Atomic numbe

...

Atomic number 31 has CN =  6 and multiplicity 27
WOFQOA
Atomic number 49 has CN =  6 and multiplicity 36
Atomic number 49 has CN =  6 and multiplicity 9
WOGNOY
Atomic number 90 has CN =  9 and multiplicity 4
Atomic number 90 has CN =  8 and multiplicity 8
WOGPAM
Atomic number 92 has CN =  9 and multiplicity 4
Atomic number 92 has CN =  8 and multiplicity 8
WOHBOL
Atomic number 29 has CN =  5 and multiplicity 8
WOJZOM
Atomic number 62 has CN =  9 and multiplicity 8
WOLHIR
Atomic number 48 has CN =  7 and multiplicity 8
Atomic number 48 has CN =  7 and multiplicity 8
Atomic number 48 has CN =  7 and multiplicity 8
Atomic number 48 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
WOMCUY
Atomic number 48 has CN =  6 and mult

Atomic number 27 has CN =  6 and multiplicity 12
Atomic number 27 has CN =  6 and multiplicity 4
Atomic number 27 has CN =  6 and multiplicity 12
XIBZUF
Atomic number 39 has CN =  7 and multiplicity 4
Atomic number 39 has CN =  7 and multiplicity 4
XICHEY
Atomic number 48 has CN =  6 and multiplicity 8
XICJIF
Atomic number 30 has CN =  5 and multiplicity 48
Atomic number 30 has CN =  6 and multiplicity 16
XICNOO01
Atomic number 40 has CN =  8 and multiplicity 48
Atomic number 40 has CN =  8 and multiplicity 24
XICNOO
Atomic number 40 has CN =  8 and multiplicity 48
Atomic number 40 has CN =  8 and multiplicity 24
XICTEL
Atomic number 48 has CN =  6 and multiplicity 1
Atomic number 48 has CN =  6 and multiplicity 2
XICTIP
Atomic number 48 has CN =  6 and multiplicity 2
Atomic number 48 has CN =  6 and multiplicity 1
XIDBUJ
Atomic number 30 has CN =  6 and multiplicity 8
XIDCEV
Atomic number 12 has CN =  6 and multiplicity 2
Atomic number 12 has CN =  6 and multiplicity 4
XIDCIZ
Atomic n

Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
YAVZUT
Atomic number 30 has CN =  5 and multiplicity 6
Atomic number 30 has CN =  5 and multiplicity 9
YAWBAC
Atomic number 30 has CN =  5 and multiplicity 6
Atomic number 30 has CN =  5 and multiplicity 9
YAWWEC
Atomic number 28 has CN =  6 and multiplicity 3
YAWWOM
Atomic number 28 has CN =  6 and multiplicity 3
YAWXED
Atomic number 48 has CN =  6 and multiplicity 4
Atomic number 48 has CN =  6 and multiplicity 4
YAXFAI
Atomic number 48 has CN =  6 and multiplicity 4
Atomic number 48 has CN =  6 and multiplicity 8
YAYSOJ
Atomic number 30 has CN =  4 and multiplicity 2
Atomic number 30 has CN =  4 and multiplicity 2
Atomic number 30 has CN =  6 and multiplicity 2
Atomic number 30 has CN =  4 and multiplicity 2
YECBAN
Atomic number 30 has CN =  5 and multiplicity 18
YEHQEK
Atomic number 30 has CN =  4 and multiplicity 2
Atomic number 30 has CN =  5 and multiplicity 2
Atomic number 30 has CN 

YUVMIO
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  6 and multiplicity 8
Atomic number 29 has CN =  5 and multiplicity 8
YUWGUV
Atomic number 30 has CN =  4 and multiplicity 8
Atomic number 11 has CN =  6 and multiplicity 4
Atomic number 11 has CN =  6 and multiplicity 4
YUXLEJ
Atomic number 47 has CN =  2 and multiplicity 18
Atomic number 13 has CN =  6 and multiplicity 6
Atomic number 47 has CN =  2 and multiplicity 18
Atomic number 13 has CN =  6 and multiplicity 6
Atomic number 47 has CN =  2 and multiplicity 18
Atomic number 13 has CN =  6 and multiplicity 6
YUYGOQ
Atomic number 48 has CN =  6 and multiplicity 2
Atomic nu

Atomic number 49 has CN =  6 and multiplicity 2
Atomic number 49 has CN =  6 and multiplicity 2
ZOHJOY
Atomic number 25 has CN =  6 and multiplicity 8
Atomic number 25 has CN =  6 and multiplicity 8
Atomic number 25 has CN =  5 and multiplicity 8
ZOHJUE
Atomic number 25 has CN =  6 and multiplicity 8
Atomic number 25 has CN =  6 and multiplicity 8
Atomic number 25 has CN =  6 and multiplicity 8
ZOPTOQ
Atomic number 29 has CN =  4 and multiplicity 12
ZOTTIN
Atomic number 48 has CN =  8 and multiplicity 16
ZOVHUQ
Atomic number 13 has CN =  6 and multiplicity 4
Atomic number 13 has CN =  6 and multiplicity 4
Atomic number 13 has CN =  6 and multiplicity 4
Atomic number 13 has CN =  6 and multiplicity 4
ZOYWOC
Atomic number 25 has CN =  6 and multiplicity 8
ZUCKIT
Atomic number 39 has CN =  4 and multiplicity 24
ZUGDEM
Atomic number 30 has CN =  4 and multiplicity 2
Atomic number 30 has CN =  4 and multiplicity 2
Atomic number 30 has CN =  6 and multiplicity 2
Atomic number 30 has CN =  5 

In [5]:
csd_reader = io.EntryReader('CSD')
current_crystal = csd_reader.crystal("BATTUP")
current_molecule = csd_reader.molecule("BATTUP")

In [134]:
CE1 = current_molecule.atoms[0]
CE1.partial_charge?